# Introdução

Neste documentos iniciamos a extração das informações e do pdf dos documentos recuperados via busca na Base Minerva.

# Parte 0: Instalações

In [1]:
# %pip install PyPDF2

# Parte 1: Obtenção dos Metadados do Minerva

In [2]:
import os
import json

In [3]:
pdf_folder_path = "./metadata_documents"
parsed_filepath = "./parsed_pdf"

## Arquivos de Metadados

In [4]:
filename_list = []
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".json") and not filename.endswith("_x.json"):
        filename_list.append(os.path.join(pdf_folder_path, filename))

filename_list = sorted(filename_list)

In [5]:
filename_list [:5]

['./metadata_documents/documento_.json',
 './metadata_documents/documento_276189.json',
 './metadata_documents/documento_494080.json',
 './metadata_documents/documento_547235.json',
 './metadata_documents/documento_614469.json']

## Arquivos Já Parseados para Ignorar

In [6]:
parsed_filename_list = []
for filename in os.listdir(parsed_filepath):
    if filename.endswith(".txt"):
        parsed_filename_list.append(os.path.join(parsed_filepath, filename))

parsed_filename_list = sorted(parsed_filename_list)

In [23]:
document_identifier = set([filename.split('/')[-1].split('.')[0] for filename in parsed_filename_list])
len(document_identifier)

1972

## Remoção de Arquivos Já Parseados

In [8]:
filename_list = [filename for filename in filename_list if filename.split('/')[-1].split('.')[0] not in document_identifier]

In [9]:
filename_list[:5]

['./metadata_documents/documento_.json',
 './metadata_documents/documento_820983.json',
 './metadata_documents/documento_841190.json',
 './metadata_documents/documento_854537.json',
 './metadata_documents/documento_884503.json']

## Obtenção dos Metadados

In [10]:
def get_metadata(filename_list):
    metadata_dict = {}
    
    for filename in filename_list:
        with open(filename, "r") as file: 
            metadata = json.load(file)
            _id = metadata["id"]
            metadata_dict[_id] = metadata
    
    return metadata_dict

In [11]:
metadata_dict = get_metadata(filename_list)

In [12]:
len(metadata_dict.keys())

78

In [13]:
selected_metadata = metadata_dict[list(metadata_dict.keys())[1]]
selected_metadata

{'title': 'Estabilização de polipropileno com antioxidantes naturais : comparação entre as técnicas de mistura in situ e no estado fundido',
 'authors': ['Oliveira, Renato Jonas Benne de.',
  'Marques, Maria de Fátima Vieira orient.',
  'Universidade Federal do Rio de Janeiro.'],
 'abstract': ' polipropileno é um polímero muito versátil o que lhe implica a capacidade de atuar em diversos setores industriais. Para isso, a arquitetura e a massa molar do polipropileno são controladas para otimizar uma determinada propriedade. Os polímeros estão sujeitos ao processo degradativo o que acarreta perda de propriedades e funcionalidade para um determinado fim. Por isso, adicionou-se ao polipropileno antioxidantes naturais e sintético para sua estabilização. Para se misturar os aditivos ao polímero foram utilizadas as técnicas de adição in situ e mistura a partir do estado fundido. Um catalisador Ziegler-Natta com controle morfológico de partículas foi preparado para a síntese do polipropileno. 

# Parte 2: Recuperação dos Arquivos PDF da URL

In [14]:
import requests
import io
import json

In [15]:
def get_pdf_file(metadata):      
    url = metadata["pdf_url"]
    _id = metadata["id"]

    response = requests.get(url)
    if response.status_code != 200:
        print("Error Requestig PDF!", _id, url)
        return None
    
    pdf_file = io.BytesIO(response.content)
    
    return pdf_file

In [16]:
pdf_file = get_pdf_file(selected_metadata)

In [17]:
pdf_file

# Parte 3: Parseamento do PDF

In [18]:
import PyPDF2

In [19]:
class PDFParser:
    
    def parse(self, pdf_file, ignore_pages: set = {}):
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        pdf_document = ""
        for idx, page in enumerate(pdf_reader.pages):
            if idx in ignore_pages:
                continue
                
            pdf_document += page.extract_text() + "\n"
        
        return pdf_document

In [20]:
pdf_parser = PDFParser()

In [21]:
def parse_pdf_file(pdf_file, metadata):
    bibliography_pages = metadata["bibliography_pages"]
    ignore_pages = set(range(bibliography_pages[0], bibliography_pages[1]+1))
    
    full_text = pdf_parser.parse(pdf_file, ignore_pages=ignore_pages)
    return full_text

In [ ]:
parsed_pdf_file = parse_pdf_file(pdf_file, selected_metadata)

In [ ]:
print(parsed_pdf_file[:500])

# Parte 4: Salvamento dos PDFs Parseados

In [ ]:
for key in metadata_dict:
    try:
        metadata = metadata_dict[key]
        
        pdf_file = get_pdf_file(metadata)
        if pdf_file is None: continue
            
        parsed_pdf_file = parse_pdf_file(pdf_file, metadata)
        
        _id = "_".join(key.split('-'))
        print(_id)
        filename = f"{parsed_filepath}/documento_" + _id + ".txt"
    
        # descomentar quando for salvar    
        with open(filename, "w") as file:
            file.write(parsed_pdf_file.encode('utf-8', 'replace').decode())

    except:
        print("Error Parsing!", key)